In [1]:
import dill
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
path_name = "../dataset/data/"

with open(path_name + 'comment-pos.data', 'rb') as file:
    datatofile = dill.load(file)

tagged_sents = []
for data in datatofile:
    text_inside = []
    for word, pos, label in data:
        text_inside.append((word, label))
    tagged_sents.append(text_inside)

train_sents, test_sents = train_test_split(tagged_sents, test_size=0.2, random_state=42)
print(len(train_sents))
print(len(test_sents))
print(train_sents[1])

552
138
[('ถ้า', 'B-c'), ('เดินทาง', 'I-c'), ('กลางคืน', 'I-c'), ('ก็', 'I-c'), ('รถทัวร์', 'I-c'), ('ครับ', 'I-c'), (' ', 'I-c'), (' ', 'O'), ('เพราะ', 'B-p'), ('รถ', 'I-p'), ('ไม่', 'I-p'), ('เยอะ', 'I-p'), (' ', 'I-p'), ('ความเสี่ยง', 'I-p'), ('การ', 'I-p'), ('เกิด', 'I-p'), ('อุบัติ', 'I-p'), ('ห', 'I-p'), ('ตุ', 'I-p'), ('ก็', 'I-p'), ('น้อย', 'I-p'), ('(', 'I-p'), ('มั้ง', 'I-p'), (')', 'I-p'), (' ', 'I-p'), (' ', 'O'), ('ถ้า', 'B-c'), ('กลางวัน', 'I-c'), ('ก็', 'I-c'), ('เครื่องบิน', 'I-c'), ('ครับ', 'I-c'), (' ', 'I-c'), (' ', 'O'), ('เพราะ', 'B-p'), (' ', 'I-p'), ('มัน', 'I-p'), ('ใช้เวลา', 'I-p'), ('น้อย', 'I-p'), ('จะ', 'I-p'), ('ได้', 'I-p'), ('มี', 'I-p'), ('เวลา', 'I-p'), ('ระหว่าง', 'I-p'), ('วัน', 'I-p'), ('เยอะ', 'I-p'), ('ๆ', 'I-p'), (' ', 'I-p')]


In [3]:
_NER_TAGS = [
        "O",
        "B_C",
        "B_P",
        "I_C",
        "I_P"
    ]

In [4]:
def convert_to_simple_transformer_format(sentences):
    sentence_id = []
    words = []
    labels = []

    for idx, sents in enumerate(sentences):
        for word, label in sents:
            label = label.upper().replace("-", "_")
            sentence_id.append(idx)
            words.append(word)
            labels.append(label)
    return pd.DataFrame(
        {"sentence_id": sentence_id, "words": words, "labels": labels}
    )    
            


In [5]:
train_ = convert_to_simple_transformer_format(train_sents)
train_

,sentence_id,words,labels
0,0,อะไหล่,B_C
1,0,เทอร์โบ,I_C
2,0,,I_C
3,0,อี,I_C
4,0,ซุ,I_C
...,...,...,...
37405,551,ทำ,I_P
37406,551,อะไร,I_P
37407,551,ได้,I_P
37408,551,หลายอย่าง,I_P


In [6]:
test_ = convert_to_simple_transformer_format(test_sents)

In [7]:
import torch
from simpletransformers.ner import NERModel, NERArgs

# Configure the model
ner_args = NERArgs()
ner_args.train_batch_size = 12
ner_args.evaluate_during_training = False
ner_args.overwrite_output_dir = True
ner_args.num_train_epochs = 100 #10


model = NERModel(
    "bert", "airesearch/wangchanberta-base-att-spm-uncased", args=ner_args, use_cuda=torch.cuda.is_available(), labels=_NER_TAGS
)

# Train the modelk
# model.train_model(train_)

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.6.attention.self.query.weight', 'roberta.encoder.layer.11.attention.self.query.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.intermediate.dense.bias', 'roberta.encoder.layer.2.intermediate.dense.weight', 'roberta.encoder.layer.1.output.dense.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.8.attention.self.key.weight', 'roberta.encoder.layer.9.attention.self.key.weight', 'roberta.encoder.layer.8.attention.self.key.bias', 'roberta.encoder.layer.2.attention.self.value.weight', 'roberta.encoder.layer.9.attention.output.LayerNorm.bi

TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [9]:
test_ner = NERModel("bert", 'weight1/checkpoint-4600-epoch-100', args=ner_args)

In [10]:
# Evaluate the model
result, model_outputs, preds_list = test_ner.eval_model(test_)
result

Running Evaluation: 100%|██████████| 18/18 [00:04<00:00,  4.38it/s]
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_C seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: I_C seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: B_P seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\pitiw\miniconda3\envs\argument\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: I_P seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 3.813645319806205,
 'precision': 0.08484848484848485,
 'recall': 0.27450980392156865,
 'f1_score': 0.12962962962962965}

In [11]:
idx = 4
test_pred = " ".join(list(map(lambda word: word[0], test_sents[idx])))
print(test_pred)
print(test_sents[idx])

หาก พิจารณา จาก การเข้าสู่ การ เป็น   AEC   ใน สิ้นปี นี้ แล้ว   ผม ว่า ความสามารถ ทาง ด้าน ภาษาอังกฤษ จะ ใช้ประโยชน์ ได้ มากกว่า คณิตศาสตร์ เยอะ มาก ครับ   ได้ ทั้ง ด้าน ติดต่อ ธุรกิจ    การท่องเที่ยว    การศึกษา
[('หาก', 'B-p'), ('พิจารณา', 'I-p'), ('จาก', 'I-p'), ('การเข้าสู่', 'I-p'), ('การ', 'I-p'), ('เป็น', 'I-p'), (' ', 'I-p'), ('AEC', 'I-p'), (' ', 'I-p'), ('ใน', 'I-p'), ('สิ้นปี', 'I-p'), ('นี้', 'I-p'), ('แล้ว', 'I-p'), (' ', 'O'), ('ผม', 'B-c'), ('ว่า', 'I-c'), ('ความสามารถ', 'I-c'), ('ทาง', 'I-c'), ('ด้าน', 'I-c'), ('ภาษาอังกฤษ', 'I-c'), ('จะ', 'I-c'), ('ใช้ประโยชน์', 'I-c'), ('ได้', 'I-c'), ('มากกว่า', 'I-c'), ('คณิตศาสตร์', 'I-c'), ('เยอะ', 'I-c'), ('มาก', 'I-c'), ('ครับ', 'I-c'), (' ', 'O'), ('ได้', 'B-p'), ('ทั้ง', 'I-p'), ('ด้าน', 'I-p'), ('ติดต่อ', 'I-p'), ('ธุรกิจ', 'I-p'), ('  ', 'I-p'), ('การท่องเที่ยว', 'I-p'), ('  ', 'I-p'), ('การศึกษา', 'I-p')]


In [15]:
# Make predictions with the model
predictions, raw_outputs = test_ner.predict([test_pred], split_on_space=True)
print(predictions[0])

Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  6.84it/s]

[{'หาก': 'B_C'}, {'พิจารณา': 'I_C'}, {'จาก': 'I_C'}, {'การเข้าสู่': 'I_C'}, {'การ': 'I_C'}, {'เป็น': 'I_C'}, {'AEC': 'I_C'}, {'ใน': 'I_C'}, {'สิ้นปี': 'I_C'}, {'นี้': 'I_C'}, {'แล้ว': 'I_C'}, {'ผม': 'B_C'}, {'ว่า': 'I_C'}, {'ความสามารถ': 'I_C'}, {'ทาง': 'I_C'}, {'ด้าน': 'I_P'}, {'ภาษาอังกฤษ': 'I_P'}, {'จะ': 'I_P'}, {'ใช้ประโยชน์': 'I_P'}, {'ได้': 'I_P'}, {'มากกว่า': 'I_P'}, {'คณิตศาสตร์': 'I_P'}, {'เยอะ': 'I_P'}, {'มาก': 'I_P'}, {'ครับ': 'I_P'}, {'ได้': 'I_P'}, {'ทั้ง': 'I_P'}, {'ด้าน': 'I_P'}, {'ติดต่อ': 'I_P'}, {'ธุรกิจ': 'I_P'}, {'การท่องเที่ยว': 'I_P'}, {'การศึกษา': 'I_P'}]


In [14]:
# Make predictions with the model
predictions, raw_outputs = test_ner.predict([test_pred], split_on_space=False)
print(predictions[0])

Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 30.30it/s]

[{'ห': 'B_C'}, {'า': 'I_C'}, {'ก': 'O'}, {' ': 'I_C'}, {'พ': 'I_C'}, {'ิ': 'I_C'}, {'จ': 'I_C'}, {'า': 'I_C'}, {'ร': 'I_C'}, {'ณ': 'I_C'}, {'า': 'I_C'}, {' ': 'O'}, {'จ': 'I_C'}, {'า': 'I_C'}, {'ก': 'I_C'}, {' ': 'I_C'}, {'ก': 'I_C'}, {'า': 'I_C'}, {'ร': 'I_C'}, {'เ': 'I_C'}, {'ข': 'I_P'}, {'้': 'I_P'}, {'า': 'I_P'}, {'ส': 'I_P'}, {'ู': 'I_P'}, {'่': 'I_P'}, {' ': 'I_P'}, {'ก': 'I_P'}, {'า': 'I_P'}, {'ร': 'I_P'}, {' ': 'I_P'}, {'เ': 'I_P'}, {'ป': 'I_P'}, {'็': 'I_P'}, {'น': 'I_P'}, {' ': 'I_P'}, {' ': 'I_P'}, {' ': 'O'}, {'A': 'I_P'}, {'E': 'I_P'}, {'C': 'I_P'}, {' ': 'I_P'}, {' ': 'I_P'}, {' ': 'O'}, {'ใ': 'O'}, {'น': 'O'}, {' ': 'O'}, {'ส': 'O'}, {'ิ': 'O'}, {'้': 'O'}, {'น': 'O'}, {'ป': 'O'}, {'ี': 'O'}, {' ': 'O'}, {'น': 'O'}, {'ี': 'O'}, {'้': 'O'}, {' ': 'O'}, {'แ': 'O'}, {'ล': 'O'}, {'้': 'O'}, {'ว': 'O'}, {' ': 'O'}, {' ': 'O'}, {' ': 'O'}, {'ผ': 'O'}, {'ม': 'O'}, {' ': 'O'}, {'ว': 'O'}, {'่': 'O'}, {'า': 'O'}, {' ': 'O'}, {'ค': 'O'}, {'ว': 'O'}, {'า': 'O'}, {'ม': 'O'}, {'ส': '

In [26]:
y_test = []
for sent in test_sents:
    labels = []
    for word, label in sent:
        if word == ' ' or word == '  ':
            continue;
        label = label.upper().replace("-", "_")
        labels.append(label)
    y_test.append(labels)
    
print(y_test[10])


['B_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'I_C', 'B_P', 'I_P', 'I_P', 'I_P', 'I_P', 'I_P', 'I_P', 'I_P', 'I_P', 'I_P', 'I_P', 'I_P', 'I_P', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [27]:
test_list = []
for sent in test_sents:
    words = []
    for word, label in sent:
        words.append(word)
    test_list.append(" ".join(words))


predictions, raw_outputs = test_ner.predict(test_list)

y_pred = []
for preds in predictions:
    y_pred.append([list(pred.items())[0][1] for pred in preds])

print(y_pred[10])

Running Prediction: 100%|██████████| 18/18 [00:02<00:00,  7.94it/s]


['O', 'I_C', 'I_C', 'I_C', 'I_P', 'I_C', 'I_C', 'O', 'I_P', 'O', 'I_P', 'I_P', 'I_P', 'O', 'I_P', 'I_P', 'I_P', 'I_P', 'O', 'O', 'O', 'I_P', 'I_P', 'I_P', 'I_P', 'I_P', 'O', 'O', 'O', 'I_P', 'I_P', 'I_P', 'I_P', 'O', 'O', 'O', 'O', 'O', 'O', 'I_P', 'I_P', 'I_P', 'I_P', 'I_C']


In [28]:
y_pred_ = []
y_test_ = []
for i in range(len(y_test)):
    if len(y_pred[i]) != len(y_test[i]):
        continue;
    y_pred_.append(y_pred[i])
    y_test_.append(y_test[i])


In [29]:
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

print("accuracy:" ,accuracy_score(y_test_, y_pred_))
print(classification_report(y_test_, y_pred_))

accuracy: 0.5009717123731375
              precision    recall  f1-score   support

          _C       0.00      0.00      0.00       128
          _P       0.01      0.05      0.01       128

   micro avg       0.01      0.02      0.01       256
   macro avg       0.00      0.02      0.01       256
weighted avg       0.00      0.02      0.01       256

